<a href="https://colab.research.google.com/github/albisbub/DCF/blob/master/_notebooks/2020-04-29-ALK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFIC MEETING 7
> Alaska Airlines Buy Pitch

- toc:true
- branch: master
- badges: true
- comments: true
- author: Blaise and Abby
- permalink: /ALK/
- categories: [fastpages, jupyter]

In [142]:
#hide
#Requirements:
import warnings
warnings.filterwarnings('ignore')
!pip install requests bs4 pandas yahoofinancials yahoo_fin requests_html
!pip install argparse
!pip install matplotlib urllib3 seaborn
!pip install fuckit

In [0]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from datetime import timedelta, datetime, date
%config InlineBackend.figure_format = 'retina'

chart_width = 550
chart_height= 400

import gspread
from oauth2client.service_account import ServiceAccountCredentials

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import requests
# !pip install 
from bs4 import BeautifulSoup
import pandas as pd
from yahoofinancials import YahooFinancials
from yahoo_fin.stock_info import get_quote_table, get_stats, get_live_price, get_income_statement, get_balance_sheet, get_cash_flow
# !pip install dict
# from dict import dict
def getQuote(ticker):
    return str(round(get_live_price(ticker), 2))

def finviz_summaryTable(symbols):
    # Get Column Header
    req = requests.get("https://finviz.com/quote.ashx?t=FB")
    soup = BeautifulSoup(req.content, 'html.parser')
    table = soup.find_all(lambda tag: tag.name=='table')
    rows = table[8].findAll(lambda tag: tag.name=='tr')
    out=[]
    for i in range(len(rows)):
        td=rows[i].find_all('td')
        out=out+[x.text for x in td]

    ls=['Ticker','Sector','Sub-Sector','Country']+out[::2]

    dict_ls={k:ls[k] for k in range(len(ls))}
    df=pd.DataFrame()
    print('Fetching Data...')
    for j in range(len(symbols)):

      try:

        req = requests.get("https://finviz.com/quote.ashx?t="+symbols[j])
        if req.status_code !=200:
            continue
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all(lambda tag: tag.name=='table')
        rows=table[6].findAll(lambda tag: tag.name=='tr')
        sector=[]
        for i in range(len(rows)):
            td=rows[i].find_all('td')
            sector=sector+[x.text for x in td]
        sector=sector[2].split('|')
        rows = table[8].findAll(lambda tag: tag.name=='tr')
        out=[]
        for i in range(len(rows)):
            td=rows[i].find_all('td')
            out=out+[x.text for x in td]
        out=[symbols[j]]+sector+out[1::2]
        out_df=pd.DataFrame(out).transpose()
        df=df.append(out_df,ignore_index=True)
        print('Done!')
        
        df=df.rename(columns=dict_ls)   

      except BaseException as E:
        print("Could not find data:", E)

    return(df)
    
def yf_summaryTable(tickerList):
    values = []
    for i in tickerList:
      try:
        value = get_quote_table(i , dict_result = True)
        values.append(value)
      except BaseException as e:
        print(e)
             
    outD = dict(zip(tickerList,values))
            
    return outD
        
def yf_stats_compTable(tickerList):
    values = []
    for i in tickerList:
      try:
        value = get_stats(i)
        values.append(value)
      except BaseException as e:
        print(e)
             
    outD= dict(zip(tickerList,values))
            
    return outD
               
def get_prices(tickerlist, start, end, period):
    yahoo_financials = YahooFinancials(tickerlist)
    jsonDATA = yahoo_financials.get_historical_price_data(start,end, period)
    return jsonDATA

In [0]:
#hide
def make_multi_LineGraph(data, logged):
  try:  
    df = data
    if logged == True:
      df['log Value'] =  np.around(np.log(df.Value), 4)
      aY = ('log Value:Q')
      scc = alt.Scale(type='log')
      color1 = alt.Color('Metric:N', legend=alt.Legend(title="Natural Log of Metrics", labelFontSize=15, titleFontSize=17),
                      scale=alt.Scale(scheme='tableau20'))
    else:
      aY = ('Value:Q')
      scc=alt.Scale(type='linear')
      color1 = alt.Color('Metric:N', legend=alt.Legend(title="Metrics",labelFontSize=15, titleFontSize=17),
                      scale=alt.Scale(scheme='tableau20'))   
    label = alt.selection_multi(fields=['Metric'], bind='legend', 
        on='mouseover',  # select on mouseover events
        nearest=True,    # select data point nearest the cursor
        empty='none'     # empty selection includes no data points
    )
    # define our base line chart of stock prices
    base = alt.Chart().mark_line().encode(
        alt.X('Year:T'),
        alt.Y(aY, scale=scc) ,
        color = color1 
        
        
        # alt.Color(' Metric:N'), opacity = alt.condition(label, alt.value(1), alt.value(0.5))

    )

    graph = alt.layer(
        base, # base line chart
        
        # add a rule mark to serve as a guide line
        alt.Chart().mark_rule(color='#aaa').encode(
            x='Year:T'
        ).transform_filter(label),
        
        # add circle marks for selected time points, hide unselected points
        base.mark_circle().encode(
            opacity=alt.condition(label, alt.value(1), alt.value(0))
        ).add_selection(label),

        # add white stroked text to provide a legible background for labels
        base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
            text=aY
        ).transform_filter(label),

        # add text labels for stock prices
        base.mark_text(align='left', dx=5, dy=-5).encode(
            text=aY
        ).transform_filter(label),
        
        data=df
    ).properties(
        width=700,
        height=400
    )
    
    return graph
  except BaseException as e:
    pass


def formatPeers(inputs):
  ColumnList = list(inputs.columns)
  yearList = list(inputs.Ticker)

  print(ColumnList)
  print(yearList)


  transposed = inputs.transpose()
  cleaned = transposed.reset_index()

  new_header = cleaned.iloc[0] #grab the first row for the header
  cleaned = cleaned[1:] #take the data less the header row
  cleaned.columns = new_header #set the header row as the df header

  byYear = cleaned.set_index("Ticker").stack()

  df = pd.DataFrame(byYear, columns=['Value'])
  df.reset_index(inplace=True)
  print(df.columns)
  df.rename(columns={0:'Ticker'}, inplace=True)
  df.dropna(inplace=True)
  df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header


  return df

def get_financials(tickerList):
  """
  list of lists
  same order as ticker list
  each list contains a list of datframes in the order income statement , balance sheet, cash flow
  """

  tickersDF = []
  for i in tickerList:

    income_statement = get_income_statement(i)
    
    balance_sheet = get_balance_sheet(i)
    
    cash_flow = get_cash_flow(i)

    finList = [income_statement, balance_sheet, cash_flow]
    tickersDF.append(finList)

  return tickersDF

def interactive_bat(data):


  source = data
  brush = alt.selection(type='interval', encodings=['x'])

  bars = alt.Chart().mark_bar().encode(
      x='Ticker:N',
      y='mean(Value):Q',
      opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
  ).add_selection(
      brush
  )

  line = alt.Chart().mark_rule(color='firebrick').encode(
      y='mean(Value):Q',
      size=alt.SizeValue(3)
  ).transform_filter(
      brush
  )

  graph = alt.layer(bars, line, data=source)

  return graph

  

In [0]:
#hide
subjectCompany = "ALK"
peers = ["AAL", "DAL", 'LUV', 'UAL']
tickerList = []
tickerList.append(subjectCompany)
for i in peers:

  tickerList.append(i)

In [0]:
#hide
all_financials_querry = get_financials(tickerList)
#Works
subjectTfinancials = all_financials_querry[0]

In [0]:
#hide
## Clean some shit up for charting
from datetime import date
today = date.today()

now = str(today.strftime("%m/%d/%Y"))
print(now)

for i in subjectTfinancials:
  a = i
  a.rename({'ttm': now}, inplace=True, axis = "columns")
  a.rename({'Breakdown': "Year"}, inplace=True, axis = "columns")


In [0]:
#hide
!pip install pyfinmod
from pyfinmod.financials import Financials
parser = Financials(subjectCompany)

# (parser.cash_flow_statement)
# (parser.balance_sheet_statement)
# (parser.income_statement)


cash = (parser.cash_flow_statement).transpose().reset_index().rename(columns={'index':'Year'})
balance = (parser.balance_sheet_statement).transpose().reset_index().rename(columns={'index':'Year'})
income = (parser.income_statement).transpose().reset_index().rename(columns={'index':'Year'})# income = subjectTfinancials[0]

# income = subjectTfinancials[0]
# balance = subjectTfinancials[1]
# cash = subjectTfinancials[2]

# Subjet Company Financials

## Income Statement

In [0]:
#hide-input
income
## LONG FORMATED FINANCIALS (Year in columns)


In [0]:
#hide
## WIDE FORMATTED FINANCIALS (Metric In Columns)

# def formatStmnt(inputs):
#   ColumnList = list(inputs.columns)
#   yearList = list(inputs.Year)

#   print(ColumnList)
#   print(yearList)


#   transposed = inputs.transpose()
#   cleaned = transposed.reset_index()

#   new_header = cleaned.iloc[0] #grab the first row for the header
#   cleaned = cleaned[1:] #take the data less the header row
#   cleaned.columns = new_header #set the header row as the df header

#   byYear = cleaned.set_index("Year").stack()

#   df = pd.DataFrame(byYear, columns=['Value'])
#   df.reset_index(inplace=True)
#   print(df.columns)
#   df.rename(columns={0:'Metric'}, inplace=True)
#   df.dropna(inplace=True)
#   df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
#   df = inputs



#   return df


def formatStmnt(inputs):
  ColumnList = list(inputs.columns)
  yearList = list(inputs.Year)

  # print(ColumnList)
  # print(yearList)


  # transposed = inputs.transpose()
  cleaned = inputs.rename_axis('Metric')
  # print(cleaned.columns)
  # cleaned.drop(cleaned.columns[0],inplace=True)
  byYear = cleaned.set_index('Year').stack()

  df = pd.DataFrame(byYear, columns=['Value'])
  df.reset_index(inplace=True)
  # print(df.columns)
  df.rename(columns={'row name':'Metric'}, inplace=True)
  # df.dropna(inplace=True)
  # df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
  # print(df)


  # print(df.columns)
  df = df[~df['Metric'].isin(['level_0','index'])]
  # df = df.drop(['level_0','index'],axis=0)
  # print(df.Year)

  df.Year = df.Year.astype(str)
  return df

In [0]:
#hide
#stack the transposed dataframes.
df = formatStmnt(income)
metricList = df[df.columns[1]].unique()

In [0]:
#hide
####PICK METRICS


columnList = [


"Revenue"  , 
"Cost of Revenue"  , 
"Gross Profit"  , 



"R&D Expenses"  , 
"SG&A Expense"  , 
"Operating Expenses"  , 
"Operating Income"  , 
"Interest Expense"  , 



"Earnings before Tax"  , 
"Income Tax Expense"  , 

"Net Income - Non-Controlling int"  , 
"Net Income - Discontinued ops"  , 
"Net Income"  , 

"Preferred Dividends"  , 
"Net Income Com"  , 

"EPS"  , 
"EPS Diluted"  , 
"Dividend per Share"  , 

"Weighted Average Shs Out"  , 


"Weighted Average Shs Out (Dil)"  , 





"Gross Margin"  , 
"EBITDA Margin"  , 
"EBIT Margin"  , 
"Profit Margin"  , 
"Free Cash Flow margin"  , 
"EBITDA"  , 
"EBIT"  , 


"Consolidated Income"  , 
"Earnings Before Tax Margin"  , 
"Net Profit Margin"  , 

"Revenue Growth"  , 

]

revenueBases = ['Revenue','Gross Profit', "Operating Income" ,"Earnings before Tax" , "Net Income", "Net Income Com","Consolidated Income" ,"EBITDA" ,"EBIT" ]
revg = ["Revenue Growth"]
epsS = ["EPS"  , 
"EPS Diluted"  , 
"Dividend per Share"  ,]
costs = [  "Cost of Revenue"  ,"Operating Expenses" ,'Interest Expense', 'Income Tax Expense', "Preferred Dividends"  , ]

oppCosts = ["R&D Expenses"  , 
"SG&A Expense"  , ]

floats = ["Weighted Average Shs Out"  , 


"Weighted Average Shs Out (Dil)"  , 
]
margins = [
"Gross Margin"  , 
"EBITDA Margin"  , 
"EBIT Margin"  , 
"Profit Margin"  , 
"Free Cash Flow margin"  , 
"Earnings Before Tax Margin"  , 
"Net Profit Margin"  , 
]

growthRates = ["Revenue Growth"]

revs_g = df[df['Metric'].isin(revenueBases)]
eps_g = df[df['Metric'].isin(epsS)]
cost_g = df[df['Metric'].isin(costs)]
operating_g = df[df['Metric'].isin(oppCosts)]
float_g = df[df['Metric'].isin(floats)]
margins_g = df[df['Metric'].isin(margins)]

revs_gg = df[df['Metric'].isin(growthRates)]



In [0]:
#hide
most_recent_float = list((float_g.Value).dropna())[0]

### Revenues

In [0]:
#hide-input


make_multi_LineGraph(revs_g, False)


### Revenue Growth & Change in Growth

In [0]:
#hide-input
make_multi_LineGraph(revs_gg, False)


In [0]:
#hide-input

make_multi_LineGraph(revs_gg, True)


### EPS

In [0]:
#hide-input

make_multi_LineGraph(eps_g, False)



### Cost Components

In [0]:
#hide-input

make_multi_LineGraph(cost_g, False)


In [0]:
#hide-input

make_multi_LineGraph(cost_g, True)


In [0]:
#hide-input

make_multi_LineGraph(operating_g, False)


In [0]:
make_multi_LineGraph(operating_g, True)


### Shares Outstanding

In [0]:
#hide-input

make_multi_LineGraph(float_g, False)

## Margins

In [0]:
make_multi_LineGraph(margins_g, False)

## Balance Sheet

In [0]:
#hide-input

balance

In [0]:
#hide
df = formatStmnt(balance)
metricList = df[df.columns[1]].unique()
for i in metricList:
  print(str("'"+i+"'"), ' ,')

In [0]:
#hide
columnList = [
'Cash and cash equivalents'  ,
'Short-term investments'  ,
'Cash and short-term investments'  ,
'Receivables'  ,
'Inventories'  ,
'Total current assets'  ,
'Property, Plant & Equipment Net'  ,
'Goodwill and Intangible Assets'  ,
'Long-term investments'  ,
'Tax assets'  ,
'Total non-current assets'  ,
'Total assets'  ,
'Payables'  ,
'Short-term debt'  ,
'Total current liabilities'  ,
'Long-term debt'  ,
'Total debt'  ,
'Deferred revenue'  ,
'Tax Liabilities'  ,
'Deposit Liabilities'  ,
'Total non-current liabilities'  ,
'Total liabilities'  ,
'Other comprehensive income'  ,
'Retained earnings (deficit)'  ,
'Total shareholders equity'  ,
'Investments'  ,
'Net Debt'  ,
'Other Assets'  ,
'Other Liabilities'  ,
]


assets = [
      
      'Cash and cash equivalents'  ,
'Short-term investments'  ,
'Cash and short-term investments'  ,
'Receivables'  ,
'Inventories'  ,
'Total current assets'  ,
'Property, Plant & Equipment Net'  ,
'Goodwill and Intangible Assets'  ,
'Tax assets'  ,
'Total non-current assets'  ,
'Total assets'  ,
'Other Assets'  ,
'Other Liabilities'  ,

      ]

depre = [   
'Long-term investments'  ,
'Investments'  ,
'Cash and short-term investments'  ,
]

current_liabs = [
'Payables'  ,
'Short-term debt'  ,
'Total current liabilities'  ,
'Long-term debt'  ,
'Total debt'  ,
'Deferred revenue'  ,
'Tax Liabilities'  ,
'Deposit Liabilities'  ,
'Total non-current liabilities'  ,
'Total liabilities'  ,


]


liab_equi = [              'Total liabilities',
'Total shareholders equity' ,]


equity = [ 'Common Stock',
              'Retained Earnings']

     

assets_g = df[df['Metric'].isin(assets)]
dep_g = df[df['Metric'].isin(depre)]
clab_g = df[df['Metric'].isin(current_liabs)]
llab_g = df[df['Metric'].isin(long_liabs)]
ls_g = df[df['Metric'].isin(long_short)]
e =  df[df['Metric'].isin(equity)]
le = df[df['Metric'].isin(liab_equi)]

# liabs_ass = df[df['Metric'].isin(la)]


### Assets

In [0]:
#hide-input

make_multi_LineGraph(assets_g, False)


In [0]:
make_multi_LineGraph(assets_g, True)


### Investment and Cash Equivalents

In [0]:
#hide-input

make_multi_LineGraph(dep_g, False)


In [0]:
make_multi_LineGraph(dep_g, True)


### Liabilities

In [0]:
#hide-input

make_multi_LineGraph(clab_g, False)

### Liabilities Vs Equity

In [0]:
#hide-input

make_multi_LineGraph(le, False)


In [0]:
#hide-input

make_multi_LineGraph(le, True)


## Cash Flows

In [0]:
#hide-input


cash

In [0]:
#hide
df = formatStmnt(cash)
metricList = df[df.columns[1]].unique()

for i in metricList:
  print(str('"'+i+'"'),',')


In [0]:
vects = [
         "Depreciation & Amortization" ,
"Stock-based compensation" ,
"Operating Cash Flow" ,
"Capital Expenditure" ,
"Acquisitions and disposals" ,
"Investment purchases and sales" ,
"Investing Cash flow" ,
"Issuance (repayment) of debt" ,
"Issuance (buybacks) of shares" ,
"Dividend payments" ,


"Financing Cash Flow" ,


"Effect of forex changes on cash" ,
"Net cash flow / Change in cash" ,
"Free Cash Flow" ,
"Net Cash/Marketcap" ,


]


outflows = [ "Depreciation & Amortization" ,
"Stock-based compensation" ,"Dividend payments" ,"Capital Expenditure" ,"Issuance (repayment) of debt" ,
"Issuance (buybacks) of shares" ,"Effect of forex changes on cash" ,]

flows = ["Operating Cash Flow" ,"Acquisitions and disposals" ,
"Investment purchases and sales" , "Investing Cash flow" ,"Financing Cash Flow" ,"Free Cash Flow" ,
]

ratios = ["Net cash flow / Change in cash" ,"Net Cash/Marketcap" ,


]

     

out_g = df[df['Metric'].isin(outflows)]
flow_g = df[df['Metric'].isin(flows)]
ratio_g = df[df['Metric'].isin(ratios)]

### Outflows





In [0]:
make_multi_LineGraph(out_g, False)

In [0]:
make_multi_LineGraph(out_g, True)

### Other Cash Flows


In [0]:
make_multi_LineGraph(flow_g, False)

In [0]:
make_multi_LineGraph(flow_g, True)

### Ratios

In [0]:
make_multi_LineGraph(ratio_g, False)

In [0]:
make_multi_LineGraph(ratio_g, True)

# Peer Comparison

In [0]:
#hide
import sys
import requests
import bs4
res = requests.get('https://en.wikipedia.org/wiki/Russell_1000_Index')
# res = requests.get("http://www.nationmaster.com/country-info/stats/Media/Internet-users")
soup = BeautifulSoup(res.content,'lxml')
a1 = soup.find_all('table')
getReturns = a1[2]
getTickers = a1[3]
# print(a1)



df1 = pd.read_html(str(getReturns))
AnnualReturns = df1[0]
# a1 = soup.find_all('table')[3]
df2 = pd.read_html(str(getTickers))
print(len(df2))
russ1kList = df2[0]



In [0]:
#hide
IWB_comps = list(russ1kList['Ticker'])

In [0]:
#hide
import random
randomSample = []

# simuls = 30


# for i in range(simuls):
#   secure_random = random.SystemRandom()
#   a = secure_random.choice(IWB_comps)
#   randomSample.append(a)
#   # print(a)
# for i in tickerList:
#   randomSample.append(i)

In [0]:
#hide
yfStatsComp = yf_summaryTable(randomSample)
sumTable = pd.DataFrame.from_records(yfStatsComp)

In [0]:
#hide
new = sumTable.transpose()
new.columns

In [0]:
#hide
vects= ['Index', 
        'P/E', 
        'EPS (ttm)', 
        'Insider Own',
        'Shs Outstand',
        'Perf Week',
        'Market Cap', 
        'Forward P/E',
        'EPS next Y',
        'Insider Trans',
        'Shs Float', 
        'Perf Month',
        'Income',
        'PEG',
        'EPS next Q',
        'Inst Own', 
        'Short Float',
        'Perf Quarter',
        'Sales',
        'P/S',
        'EPS this Y',
        'Inst Trans',
        'Short Ratio',
        'Perf Half Y',
        'Book/sh',
        'P/B',
        'ROA',
        'Target Price',
        'Perf Year',
        'Cash/sh',
        'P/C',
        'EPS next 5Y',
        'ROE',
        '52W Range',
        'Perf YTD', 
        'Dividend', 
        'P/FCF',
        'EPS past 5Y', 
        'ROI',
        '52W High',
        'Beta', 
        'Dividend %',
        'Quick Ratio',
        'Sales past 5Y',
        'Gross Margin',
        '52W Low',
        'ATR',
        'Employees',
        'Current Ratio',
        'Sales Q/Q',
        'Oper. Margin',
        'RSI (14)',
        'Volatility',
        'Optionable',
        'Debt/Eq',
        'EPS Q/Q', 
        'Profit Margin',
        'Rel Volume',
        'Prev Close',
        'Shortable',
        'LT Debt/Eq',
        'Earnings',
        'Payout',
        'Avg Volume',
        'Price',
        'Recom', 
        'SMA20',
        'SMA50',
        'SMA200',
        'Volume',
        'Change']

In [0]:
#hide
!pip install finviz
import finviz
recs = []
xyz= 0
for i in tickerList:
    try:
      hold = [i]
      for b in (finviz.get_stock(i)):
        colls = (list((finviz.get_stock(i)).keys()))
        hold.append(finviz.get_stock(i)[b])
      recs.append(hold)
    except BaseException as e:
      print(e, "could not find", i)
print(colls)


In [0]:
#hide
listCols = (list(['Ticker']))
for i in colls:
  # print(i)
  listCols.append(i)

print(listCols)
bigTable = pd.DataFrame.from_records(recs, columns=listCols)


## Big Comp Table

In [0]:
#hide-input


bigTable

In [0]:
#hide
# import numpy as np 
# from pandas import DataFrame
# import matplotlib.pyplot as plt


# plt.pcolor(bigTable)
# plt.yticks(np.arange(0.5, len(df.index), 1), bigTable.index)
# plt.xticks(np.arange(0.5, len(df.columns), 1), bigTable.columns)
# plt.show()

In [0]:
  # hide
  # new_header = df.iloc[0] #grab the first row for the header
  # df = df[1:] #take the data less the header row
  # df.columns = new_header #set the header row as the df header 

In [0]:
#hide
#PE data http://www.stern.nyu.edu/~adamodar/pc/datasets/pedata.xls
#
#
#
#

Market Value

Enterprise Value

Leverage

# DCF Valuation

In [0]:
#hide
!pip install pyfinmod

In [0]:
#hide


from pyfinmod.financials import Financials
from pyfinmod.ev import fcf, enterprise_value

from pyfinmod.financials import Financials

from pyfinmod.financials import Financials
from pyfinmod.wacc import total_debt

from pyfinmod.financials import Financials
from pyfinmod.wacc import tax_rate





from pyfinmod.financials import Financials
from pyfinmod.wacc import cost_of_debt

from pyfinmod.financials import Financials
from pyfinmod.wacc import cost_of_equity

# RFR = .01
# mkt_return = .09
# LR_g =.1
# SR_g = .1 + LR_g

# print((RFR,mkt_return,SR_g,LR_g))




# lec = (new.loc['Weighted Average Shs Out'][0])

# eq = parser.balance_sheet_statement.loc['Total liabilities'][0]




# Calculate WACC


parser = Financials(subjectCompany)
from pyfinmod.wacc import wacc

from pyfinmod.ev import dcf, fcf
from pyfinmod.financials import Financials

def fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, shock, subjectCompany=subjectCompany):
  # print(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g))
  parser = Financials(subjectCompany)
  lec = (parser.balance_sheet_statement.iloc[-5::5])

  for i in lec.itertuples():
    ec = (i._1)
    break


  parser = Financials(subjectCompany)
  parser.income_statement.columns
  currentPrice = parser.price


  parser = Financials(subjectCompany)
  ev = enterprise_value(parser.balance_sheet_statement)[0]
  ecToev = ec / ev
  equitys_share = (1-ecToev)






  parser = Financials(subjectCompany)
  fcf_value = fcf(parser.cash_flow_statement)
  total_debt(parser.balance_sheet_statement)
  tax_rate(parser.income_statement)
  parser.mktCap



  # cost_of_debt(parser.balance_sheet_statement, parser.income_statement)

  # cost_of_equity(float(parser.beta), risk_free_interest_rate = RFR, market_return = mkt_return)
  parser = Financials(subjectCompany)

  aapl_wacc = wacc(parser.mktCap,
                  parser.balance_sheet_statement,
                  parser.income_statement,
                  float(parser.beta),
                  risk_free_interest_rate=RFR,
                  market_return=mkt_return)
  

  parser = Financials(subjectCompany)
  if shock:
    output = dcf_n1Shock(fcf(parser.cash_flow_statement), aapl_wacc, shock, short_term_growth=SR_g, long_term_growth=LR_g)
  else:
    output = dcf(fcf(parser.cash_flow_statement), aapl_wacc, short_term_growth=SR_g, long_term_growth=LR_g)
  
  value_per_share = output / (parser.income_statement.loc['Weighted Average Shs Out'][0])

  pps = equitys_share * value_per_share

  IR = ((pps - currentPrice) / currentPrice)*100

  out = (value_per_share, pps, SR_g, LR_g, IR)
  return (out)
  # return float(value_per_share)


Generating a range of growth rate scenarios:

In [0]:
#collapse-hide
SR_g_scenarios = np.linspace(-.03,.04,15,endpoint=False)
print(SR_g_scenarios)


### DCF Growth Rate Scenario

In [0]:
#hide
parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_1 = [colz]
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
for i in range(len(SR_g_scenarios)):
  SR_g = SR_g_scenarios[i]
  LR_g = .01
  records_dcf_1.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, False))
  # print(records_dcf_1)
  print("Done computing ", (i + 1), ' DCF out of ',len(SR_g_scenarios))

dcf_growth_rates_outputs = pd.DataFrame.from_records(records_dcf_1, columns = list(records_dcf_1[0]))



In [0]:
#hide-input

ind = (dcf_growth_rates_outputs.columns[4])
dcf_growth_rates_outputs

Seems to do a decent job, however this does not take into account the shock from coronavirus which will probably hurt revenues next year. We will modifiy the discounted cash flow formula to include a shock to FCF in the first year.

In [0]:
#hide
from pyfinmod.basic import npv
from math import sqrt

def dcf_n1Shock(fcfs, wacc, SHOCK, short_term_growth, long_term_growth):
    latest_fcf_date = fcfs.index.max()
    dates = pd.date_range(latest_fcf_date, periods=6, freq="365D")[1:]
    future_cash_flows = [fcfs[latest_fcf_date]]
    for i in range(5): # 5?
        # print(i)
        # print(SHOCK)
        if i == 0:        
          next_year_fcf = future_cash_flows[-1] * (1 + short_term_growth)
          future_cash_flows.append(next_year_fcf*SHOCK)

        else:
            
          next_year_fcf = future_cash_flows[-1] * (1 + short_term_growth)
          future_cash_flows.append(next_year_fcf)



    future_cash_flows = future_cash_flows[1:]
    df = pd.DataFrame(data={"fcf": future_cash_flows, "date": dates})
    # df.set_index('date', inplace=True)
    df["terminal value"] = 0
    last_index = df.index[-1]
    last_short_term_fcf = df.at[last_index, "fcf"]
    df.at[last_index, "terminal value"] = (
        last_short_term_fcf * (1 + long_term_growth) / (wacc - long_term_growth)
    )
    df["cash flow"] = df[["fcf", "terminal value"]].apply(sum, axis=1)
    return npv(df, wacc) * sqrt((1 + wacc))

parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_1 = [colz]
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
for i in range(len(SR_g_scenarios)):
  SR_g = SR_g_scenarios[i]
  LR_g = .01
  records_dcf_1.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, 1/2))
  # print(records_dcf_1)
  print("Done computing ", (i + 1), ' DCF out of ',len(SR_g_scenarios))

dcf_growth_rates_outputs_shock = pd.DataFrame.from_records(records_dcf_1, columns = list(records_dcf_1[0]))



In [0]:
#hide-input
dcf_growth_rates_outputs_shock

## DCF Peer Comparison

In [0]:
#hide
parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_2 = []
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
LR_g = .01
SR_g = .02
for i in tickerList:
  records_dcf_2.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, .5, i))
  # print(records_dcf_1)


peer_dcf = pd.DataFrame.from_records(records_dcf_2, columns = list(records_dcf_1[0]))

peer_dcf['ticker'] = tickerList
peer_dcf.set_index('ticker', inplace=True)

In [0]:
#hide-input
peer_dcf

## Historical DCF Scenario Backtest Analysis Using Differint Bases (Work In Progress.)

(WIP -- NEED TO DO MORE CODE BUT THIS WILL HOW HOW ACCURATE OUR MODELS ARE OVER TIME AND WHAT EARNINGS BASE MAKE THE BEST ESTIMATORS FOR THIS FIRM)

In [0]:
from google.colab import drive
import sys



In [0]:
#hide
'''
Utilizing financialmodelingprep.com for their free-endpoint API
to gather company financials.

NOTE: Some code taken directly from their documentation. See: https://financialmodelingprep.com/developer/docs/. 
'''

from urllib.request import urlopen
import json

def get_jsonparsed_data(url):
    '''
    Fetch url, return parsed json. 

    args:
        url: the url to fetch.
    
    returns:
        parsed json
    '''
    response = urlopen(url)
    data = response.read().decode('utf-8')
    return json.loads(data)

def get_EV_statement(ticker, period = 'annual'):
    '''
    Fetch EV statement, with details like total shares outstanding, from FMP.com

    args:
        ticker: company tickerr
    returns:
        parsed EV statement
    '''
    if period == 'annual':
        url = 'https://financialmodelingprep.com/api/v3/enterprise-value/{}'.format(ticker)
    elif period == 'quarter':
        url = 'https://financialmodelingprep.com/api/v3/enterprise-value/{}?period=quarter'.format(ticker)
    return get_jsonparsed_data(url)

#! TODO: maybe combine these with argument flag for which statement, seems pretty redundant tbh
def get_income_statement(ticker, period = 'annual'):
    '''
    Fetch income statement.

    args:
        ticker: company ticker.
        period: annual default, can fetch quarterly if specified. 

    returns:
        parsed company's income statement
    '''
    if period == 'annual':
        url = 'https://financialmodelingprep.com/api/v3/financials/income-statement/{}'.format(ticker)
    elif period == 'quarter':
        url = 'https://financialmodelingprep.com/api/v3/financials/income-statement/{}?period=quarter'.format(ticker)
    else:
        raise ValueError("in get_income_statement: invalid period")     # may as well throw...

    return get_jsonparsed_data(url)


def get_cashflow_statement(ticker, period = 'annual'):
    '''
    Fetch cashflow statement.

    args:
        ticker: company ticker.
        period: annual default, can fetch quarterly if specified. 

    returns:
        parsed company's cashflow statement
    '''
    if period == 'annual':
        url = 'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{}'.format(ticker)
    elif period == 'quarter':
        url = 'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{}?period=quarter'.format(ticker)
    else:
        raise ValueError("in get_cashflow_statement: invalid period")     # may as well throw...

    return get_jsonparsed_data(url)

def get_balance_statement(ticker, period = 'annual'):
    '''
    Fetch balance sheet statement.

    args:
        ticker: company ticker.
        period: annual default, can fetch quarterly if specified. 

    returns:
        parsed company's balance sheet statement
    '''
    if period == 'annual':
        url = 'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{}'.format(ticker)
    elif period == 'quarter':
        url = 'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{}?period=quarter'.format(ticker)
    else:
        raise ValueError("in get_balancesheet_statement: invalid period")     # may as well throw...

    return get_jsonparsed_data(url)

def get_stock_price(ticker):
    '''
    Fetches the stock price for a ticker

    args:
        ticker
    
    returns:
        {'symbol': ticker, 'price': price}
    '''
    url = 'https://financialmodelingprep.com/api/v3/stock/real-time-price/{}'.format(ticker)
    return get_jsonparsed_data(url)

def get_batch_stock_prices(tickers):
    '''
    Fetch the stock prices for a list of tickers.

    args:
        tickers: a list of  tickers........
    
    returns:
        dict of {'ticker':  price}
    '''
    prices = {}
    for ticker in tickers:
        prices[ticker] = get_stock_price(ticker)['price']

    return prices

def get_historical_share_prices(ticker, dates):
    '''
    Fetch the stock price for a ticker at the dates listed.

    args:
        ticker: a ticker.
        dates: a list of dates from which to fetch close price.

    returns:
        {'date': price, ...}
    '''
    prices = {}
    for date in dates:
        date_start, date_end = date[0:8] + str(int(date[8:]) - 2), date
        url = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to={}'.format(ticker, date_start, date_end)
        try:
            prices[date_end] = get_jsonparsed_data(url)['historical'][0]['close']
        except IndexError:
            #  RIP nested try catch, so many issues with dates just try a bunch and get within range of earnings release
            try:
                prices[date_start] = get_jsonparsed_data(url)['historical'][0]['close']
            except IndexError:
                print(date + ' ', get_jsonparsed_data(url))

    return prices

In [0]:
#hide
import argparse
from decimal import Decimal


def DCF(ticker, ev_statement, income_statement, balance_statement, cashflow_statement, discount_rate, forecast, earnings_growth_rate, cap_ex_growth_rate, perpetual_growth_rate):
    '''
    a very basic 2-stage DCF implemented for learning purposes.
    see enterprise_value() for details on arguments. 

    args:
        see enterprise value for more info...

    returns:
        dict: {'share price': __, 'enterprise_value': __, 'equity_value': __, 'date': __}
        CURRENT DCF VALUATION. See historical_dcf to fetch a history. 

    '''
    enterprise_val = enterprise_value(income_statement,
                                        cashflow_statement,
                                        balance_statement,
                                        forecast, 
                                        discount_rate,
                                        earnings_growth_rate, 
                                        cap_ex_growth_rate, 
                                        perpetual_growth_rate)

    equity_val, share_price = equity_value(enterprise_val,
                                           ev_statement)

    print('\nEnterprise Value for {}: ${}.'.format(ticker, '%.2E' % Decimal(str(enterprise_val))), 
              '\nEquity Value for {}: ${}.'.format(ticker, '%.2E' % Decimal(str(equity_val))),
           '\nPer share value for {}: ${}.\n'.format(ticker, '%.2E' % Decimal(str(share_price))),
            '-'*60)

    return {
        'date': income_statement[0]['date'],       # statement date used
        'enterprise_value': enterprise_val,
        'equity_value': equity_val,
        'share_price': share_price
    }

def historical_DCF(ticker, years, forecast, discount_rate, earnings_growth_rate, cap_ex_growth_rate, perpetual_growth_rate, interval = 'annual'):
    '''
    Wrap DCF to fetch DCF values over a historical timeframe, denoted period. 

    args:
        same as DCF, except for
        period: number of years to fetch DCF for

    returns:
        {'date': dcf, ..., 'date', dcf}
    '''
    dcfs = {}

    income_statement = get_income_statement(ticker = ticker, period = interval)['financials'] 
    balance_statement = get_balance_statement(ticker = ticker, period = interval)['financials']
    cashflow_statement = get_cashflow_statement(ticker = ticker, period = interval)['financials']
    enterprise_value_statement = get_EV_statement(ticker = ticker, period = interval)['enterpriseValues']

    if interval == 'quarter':
        intervals = years * 4
    else:
        intervals = years

    for interval in range(0, intervals):
        try:
            dcf = DCF(ticker, 
                    enterprise_value_statement[interval],
                    income_statement[interval:interval+2],        # pass year + 1 bc we need change in working capital
                    balance_statement[interval:interval+2],
                    cashflow_statement[interval:interval+2],
                    discount_rate,
                    forecast, 
                    earnings_growth_rate,  
                    cap_ex_growth_rate, 
                    perpetual_growth_rate)
        except IndexError:
            print('Interval {} unavailable, no historical statement.'.format(interval)) # catch
        dcfs[dcf['date']] = dcf 
    
    return dcfs

def ulFCF(ebit, tax_rate, non_cash_charges, cwc, cap_ex):
    '''
    Formula to derive unlevered free cash flow to firm. Used in forecasting.

    args:
        ebit: Earnings before interest payments and taxes.
        tax_rate: The tax rate a firm is expected to pay. Usually a company's historical effective rate.
        non_cash_charges: Depreciation and amortization costs. 
        cwc: Annual change in net working capital.
        cap_ex: capital expenditures, or what is spent to maintain zgrowth rate.

    returns:
        unlevered free cash flow
    '''
    return ebit * (1-tax_rate) + non_cash_charges + cwc + cap_ex

def get_discount_rate():
    '''
    Calculate the Weighted Average Cost of Capital (WACC) for our company.
    Used for consideration of existing capital structure.

    args:
    
    returns:
        W.A.C.C.
    '''
    return .1 # TODO: implement 

def equity_value(enterprise_value, enterprise_value_statement):
    '''
    Given an enterprise value, return the equity value by adjusting for cash/cash equivs. and total debt.

    args:
        enterprise_value: (EV = market cap + total debt - cash), or total value
        enterprise_value_statement: information on debt & cash
    
    returns:
        equity_value: (enterprise value - debt + cash)
        share_price: equity value/shares outstanding
    '''
    equity_val = enterprise_value - enterprise_value_statement['+ Total Debt'] 
    equity_val += enterprise_value_statement['- Cash & Cash Equivalents']
    share_price = equity_val/float(enterprise_value_statement['Number of Shares'])

    return equity_val,  share_price

def enterprise_value(income_statement, cashflow_statement, balance_statement, period, discount_rate, earnings_growth_rate, cap_ex_growth_rate, perpetual_growth_rate):
    '''
    Calculate enterprise value by NPV of explicit _period_ free cash flows + NPV of terminal value,
    both discounted by W.A.C.C.

    args:
        ticker: company for forecasting
        period: years into the future
        earnings growth rate: assumed growth rate in earnings, YoY
        cap_ex_growth_rate: assumed growth rate in cap_ex, YoY
        perpetual_growth_rate: assumed growth rate in perpetuity for terminal value, YoY

    returns:
        enterprise value
    '''
    # XXX: statements are returned as historical list, 0 most recent
    ebit = float(income_statement[0]['EBIT'])
    tax_rate = float(income_statement[0]['Income Tax Expense']) /  \
               float(income_statement[0]['Earnings before Tax'])
    non_cash_charges = float(cashflow_statement[0]['Depreciation & Amortization'])
    cwc = (float(balance_statement[0]['Total assets']) - float(balance_statement[0]['Total non-current assets'])) - \
          (float(balance_statement[1]['Total assets']) - float(balance_statement[1]['Total non-current assets']))
    cap_ex = float(cashflow_statement[0]['Capital Expenditure'])
    discount = discount_rate

    flows = []

    # Now let's iterate through years to calculate FCF, starting with most recent year
    print('Forecasting flows for {} years out, starting at {}.'.format(period, income_statement[0]['date']),
         ('\n         DFCF   |    EBIT   |    D&A    |    CWC     |   CAP_EX   | '))
    for yr in range(1, period+1):    

        # increment each value by growth rate
        ebit = ebit * (1 + (yr * earnings_growth_rate))
        non_cash_charges = non_cash_charges * (1 + (yr * earnings_growth_rate))
        cwc = cwc * 0.7                             # TODO: evaluate this cwc rate? 0.1 annually?
        cap_ex = cap_ex * (1 + (yr * cap_ex_growth_rate))         

        # discount by WACC
        flow = ulFCF(ebit, tax_rate, non_cash_charges, cwc, cap_ex)
        PV_flow = flow/((1 + discount)**yr)
        flows.append(PV_flow)

        print(str(int(income_statement[0]['date'][0:4]) + yr) + '  ',
              '%.2E' % Decimal(PV_flow) + ' | ',
              '%.2E' % Decimal(ebit) + ' | ',
              '%.2E' % Decimal(non_cash_charges) + ' | ',
              '%.2E' % Decimal(cwc) + ' | ',
              '%.2E' % Decimal(cap_ex) + ' | ')

    NPV_FCF = sum(flows)
    
    # now calculate terminal value using perpetual growth rate
    final_cashflow = flows[-1] * (1 + perpetual_growth_rate)
    TV = final_cashflow/(discount - perpetual_growth_rate)
    NPV_TV = TV/(1+discount)**(1+period)

    return NPV_TV+NPV_FCF



In [0]:
'''
Quick visualization toolkit. I'd like to build this out to be decently powerful
in terms of enabling quick interpretation of DCF related data.
'''
import sys

import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('..')

sns.set()
sns.set_context('paper')


def visualize(dcf_prices, current_share_prices, regress = True):
    '''
    2d plot comparing dcf-forecasted per share price with
    where a list of stocks is currently trading

    args:
        dcf_prices: dict of {'ticker': price, ...} for dcf-values
        current_share_prices: dict of {'ticker': price, ...} for (guess)
        regress: regress a line of best fit, because why not

    returns:
        nada
    '''
    # TODO: implement
    return NotImplementedError

def visualize_bulk_historicals(dcfs, ticker, condition):
    '''
    multiple 2d plot comparing historical DCFS of different growth
    assumption conditions

    args:
        dcfs: list of dcfs of format {'value1', {'year1': dcf}, ...}
        condition: dict of format {'condition': [value1, value2, value3]}

    '''
    dcf_share_prices = {}
    variable = list(condition.keys())[0]
    
    #TODO: make this more eloquent for handling the plotting of multiple condition formats
    try:
        conditions = [str(cond) for cond in list(condition.values())[0]]
    except IndexError:
        print(condition)
        conditions = [condition['Ticker']]

    for cond in conditions:
        dcf_share_prices[cond] = {}
        years = dcfs[cond].keys()
        for year in years:
            dcf_share_prices[cond][year] = dcfs[cond][year]['share_price']

    for cond in conditions:
        plt.plot(list(dcf_share_prices[cond].keys())[::-1], 
                 list(dcf_share_prices[cond].values())[::-1], label = cond)

    # sorry for anybody reading this, bit too pythonic
    # the second argument here just fetches the list of dates we're using as x values
    # in the above plt.plot() call without knowing the conditions we index with abo
    historical_stock_prices = get_historical_share_prices(ticker, 
                                                          list(dcf_share_prices[list(dcf_share_prices.keys())[0]].keys())[::-1])
    plt.plot(list(historical_stock_prices.keys()),
             list(historical_stock_prices.values()), label = '${} over time'.format(ticker))
    plt.rcParams["figure.figsize"]=20,20
    plt.xlabel('Date')
    plt.ylabel('Share price ($)')
    plt.legend(loc = 'upper right')
    plt.title('$' + ticker + '  ')
    return plt

def visualize_historicals(dcfs):
    '''
    2d plot comparing dcf history to share price history
    '''
    pass

    dcf_share_prices = {}
    for k, v in dcfs.items():
        dcf_share_prices[dcfs[k]['date']] = dcfs[k]['share_price']

    xs = list(dcf_share_prices.keys())[::-1]
    ys = list(dcf_share_prices.values())[::-1]

    plt.scatter(xs, ys)
    plt.rcParams["figure.figsize"]=20,20

    return plt


In [0]:
def prettyprint(dcfs, years):
    '''
    Pretty print-out results of a DCF query.
    Handles formatting for all output variatisons.
    '''
    
    LISTTEST = []

    if years > 1:
        for k, v in dcfs.items():
            LISTTEST.append('ticker: {}'.format(k))
            if len(dcfs[k].keys()) > 1:
                for yr, dcf in v.items():
                     LISTTEST.append('date: {} \
                        \nvalue: {}'.format(yr, dcf))
    else:
        for k, v in dcfs.items():
             LISTTEST.append('ticker: {}  \
                  \nvalue: {}'.format(k, v))
             
    print(LISTTEST)
    # return None
    
    


In [0]:
# !pip install argparse
# # !rm -R DCF
# !pip install matplotlib urllib3 seaborn
# # !git clone https://github.com/albisbub/DCF.git
# # !pip install git+https://github.com/albisbub/DCF.git

# # !git merge DCF/DCF
# # !ls DCF
# # !pip install git+file://c:/DCF/modelling
# # !pip install git+file://c:/DCF/visualization

# !pip install fuckit
# import fuckit
# import os
# import subprocess
# import argparse

# from modeling.data import *
# from modeling.dcf import *
# from visualization.plot import *
# from visualization.printouts import *




def main(args):
    '''
    although the if statements are less than desirable, it allows rapid exploration of 
    historical or present DCF values for either a single or list of tickers.
    '''



    parser = argparse.ArgumentParser()

    parser.add_argument('--p', '--period', help = 'years to forecast', type = int, default =  5)
    parser.add_argument('--t', '--ticker', help = 'pass a single ticker to do historical DCF', type = str, default = 'AAPL')
    parser.add_argument('--y', '--years', help = 'number of years to compute DCF analysis for', type = int, default = 1)
    parser.add_argument('--i', '--interval', help = 'interval period for each calc, either "annual" or "quarter"', default = 'annual')
    parser.add_argument('--s', '--step_increase', help = 'specify step increase for EG, CG, PG to enable comparisons.', type = float, default = 0)
    parser.add_argument('--steps', help = 'steps to take if --s is > 0', default = 5)
    parser.add_argument('--v', '--variable', help = 'if --step_increase is specified, must specifiy variable to increase from: [earnings_growth_rate, discount_rate]', default = None)
    parser.add_argument('--d', '--discount_rate', help = 'discount rate for future cash flow to firm', default = 0.1)
    parser.add_argument('--eg', '--earnings_growth_rate', help = 'growth in revenue, YoY',  type = float, default = .05)
    parser.add_argument('--cg', '--cap_ex_growth_rate', help = 'growth in cap_ex, YoY', type = float, default = 0.045)
    parser.add_argument('--pg', '--perpetual_growth_rate', help = 'for perpetuity growth terminal value', type = float, default = 0.05)


    args = parser.parse_args(args.split())

    if args.s > 0:
        if args.v is not None:
            if args.v == 'eg' or 'earnings_growth_rate':
                cond, dcfs = run_setup(args, variable = 'eg')
            elif args.v == 'cg' or 'cap_ex_growth_rate':
                cond, dcfs = run_setup(args, variable = 'cg')
            elif args.v == 'pg' or 'perpetual_growth_rate':
                cond, dcfs = run_setup(args, variable = 'pg')
            elif args.v == 'discount_rate' or 'discount':
                cond, dcfs = run_setup(args, variable = 'discount')
            # TODO: more dynamically  do this...potentially? 
            else:
                raise ValueError('args.variable is invalid, must choose (as of now) from this list -> [earnings_growth_rate, cap_ex_growth_rate, perpetual_growth_rate, discount')
        else:
            # should  we just default to something?
            raise ValueError('If step (-- s) is > 0, you must specify the variable via --v. What was passed is invalid.')
    else:
        cond, dcfs = {'Ticker': [args.t]}, {}
        dcfs[args.t] = historical_DCF(args.t, args.y, args.p, args.d, args.eg, args.cg, args.pg, args.i)

    if args.y > 1: # can't graph single timepoint very well....
        return visualize_bulk_historicals(dcfs, args.t, cond)
    else:
        prettyprint(dcfs, args.y)

def run_setup(args, variable):


  
    dcfs, cond = {}, {args.v: []}
    
    for increment in range(1, int(args.steps) + 1): # default to 5 steps?
        # this should probably be wrapped in another function..
        var = vars(args)[variable] * (1 + (args.s * increment))
        step = '{}: {}'.format(args.v, str(var)[0:4])

        cond[args.v].append(step)
        vars(args)[variable] = var
        dcfs[step] = historical_DCF(args.t, args.y, args.p, args.d, args.eg, args.cg, args.pg, args.i)

    return cond, dcfs

def multiple_tickers():
    '''
    can be called from main to spawn dcf/historical dcfs for 
    a list of tickers TODO: fully fix
    '''
    # if args.ts is not None:
    #     '''list to forecast'''
    #     if args.y > 1:
    #         for ticker in args.ts:
    #             dcfs[ticker] =  historical_DCF(args.t, args.y, args.p, args.eg, args.cg, args.pgr)
    #     else:
    #         for ticker in args.tss:
    #             dcfs[ticker] = DCF(args.t, args.p, args.eg, args.cg, args.pgr)
    # elif args.t is not None:
    #     ''' single ticker'''
    #     if args.y > 1:
    #         dcfs[args.t] = historical_DCF(args.t, args.y, args.p, args.eg, args.cg, args.pgr)
    #     else:
    #         dcfs[args.t] = DCF(args.t, args.p, args.eg, args.cg, args.pgr)
    # else:
    #     raise ValueError('A ticker or list of tickers must be specified with --ticker or --tickers')
    return NotImplementedError




### Scenario Callibration

In [0]:
#hide-input

# %%capture%%capture
%matplotlib inline

# import imp 
# !pwd
# helper = imp.new_module('helper')
# exec(open("/main.py").read(), helper.__dict__)

# with fuckit:
  #	can compute DCFs historically on either an 'annual' or 'quarter' basis. if quarter is indicated, total number of DCFS = years * 4
interval = 'quarter'
lookBackPeriod = str(3)
earnings_growth = str(.15)
#some sensitivity analysis: if this is specified, DCFs will be computed for default + (step_increase * interval_number), 
#showing specifically how changing the underlying assumption impacts valuation
step_increase = str(0.1)

#number of steps to take (for step_increase)
steps = str(2)
#the variable to increase each step, those available are: earnings_growth_rate, cap_ex_growth_rate, perpetual_growth_rate, discount_rate
variable = "earnings_growth_rate"
command = str("--t "+subjectCompany+" --i "+interval+"  --y "+lookBackPeriod+" --eg "+earnings_growth+" --steps "+steps+" --s "+step_increase+" --v "+variable+"")

a = main(command)


In [0]:
import pandas as pd
%matplotlib inline
a.show()

# Conclusions and Summary

In [0]:
# print(returned_output.decode("utf-8"))